### gets notebook name and commit hash

In [5]:
%%javascript
IPython.notebook.kernel.execute('nb_name = "' + IPython.notebook.notebook_name + '"')

<IPython.core.display.Javascript object>

In [6]:
nb_name = nb_name.rsplit('.')[0]

commit = !git rev-parse HEAD
commit = commit[0]

model_name = "{}_{}".format(nb_name, commit)

In [7]:
output_filename = "{}.txt".format(model_name)

import os

output_filepath = os.path.join('../data/models', output_filename)

model_temp_dir = os.path.join('../tmp/', model_name)

bleu_all_cat = os.path.join(model_temp_dir, "{}_all-cat.txt".format(model_name))

if not os.path.isdir(model_temp_dir):
    os.mkdir(model_temp_dir)

# Let's generate the baseline

In [8]:
import os

os.sys.path.insert(0, '../script')

from webnlg import WebNLGCorpus
from lexicalization import preprocess_so
import pandas as pd

pd.set_option('max_colwidth', 100)

test = WebNLGCorpus.load("test_with_lex")

In [60]:
def join_triples(g):
    
    datas = g[['m_subject', 'm_predicate', 'm_object']].to_dict(orient='records')
    
    sens = []
    
    for data in datas:
    
        data = {'m_subject': preprocess_so(data['m_subject']),
                'm_predicate': preprocess_so(data['m_predicate']),
                'm_object': preprocess_so(data['m_object'])
               }
    
        sen = f'{data["m_subject"]} {data["m_predicate"]} {data["m_object"]}'
        
        sens.append(sen)
        
    return ' '.join(sens)


baseline = test.mdf.groupby('idx').apply(join_triples).to_frame().reset_index()
baseline.rename(columns={0: 'text'}, inplace=True)
baseline['id'] = baseline.idx.str.split('_').apply(lambda v: v[1]).astype('int')

In [61]:
import codecs

with codecs.open(output_filepath, 'w', 'utf-8') as f:
    
    for _, text in baseline.sort_values('id').text.iteritems():
        
        f.write(text)
        f.write('\n')

In [62]:
!head "$output_filepath"

Abilene Regional Airport city Served Abilene, Texas
Adolfo Suárez Madrid–Barajas Airport location Madrid, Paracuellos de Jarama, San Sebastián de los Reyes and Alcobendas
Adolfo Suárez Madrid–Barajas Airport runway Name 18L/36R
Afonso Pena International Airport ICAO Location Identifier SBCT
Afonso Pena International Airport city Served Curitiba
Al-Taqaddum Air Base city Served Fallujah
Al-Taqaddum Air Base runway Length 3684.0
Alderney Airport runway Name 14/32
Allama Iqbal International Airport runway Length 3360.12
Amsterdam Airport Schiphol 1st runway Number 18


In [63]:
!python ../evaluation/webnlg2017/webnlg-automatic-evaluation-v2/evaluation_v2.py --team_name "$model_name" --team_filepath "$output_filepath" --outdir "$model_temp_dir"

Files creating finished for:  4 - Baseline - Just join triples_c952c4774651e01f72450b3ac8ebcde98ab9f157


In [64]:
!../evaluation/webnlg2017/webnlg-baseline-master/multi-bleu.perl -lc ../evaluation/webnlg2017/webnlg-automatic-evaluation/references/gold-all-cat-reference0.lex ../evaluation/webnlg2017/webnlg-automatic-evaluation/references/gold-all-cat-reference1.lex ../evaluation/webnlg2017/webnlg-automatic-evaluation/references/gold-all-cat-reference2.lex < "$bleu_all_cat"

BLEU = 24.83, 64.6/36.0/19.5/11.2 (BP=0.930, ratio=0.932, hyp_len=38801, ref_len=41636)
